In [7]:
import pandas as pd
import geopandas as gpd
import numpy as np
import csv
import requests
from zipfile import ZipFile
import os
from datetime import date, datetime

In [8]:
from arcgis.gis import GIS
from pathlib import Path
from zipfile import ZipFile

In [9]:
yearDate = date.today().strftime('%m%d%y')
print(str(datetime.now()) + " Running 1_COVID_Daily_Cases_Process")
print(yearDate)

2020-11-15 12:55:07.514626 Running 1_COVID_Daily_Cases_Process
111520


In [10]:
daily_covid_county_service_id = 'a7887f1940b34bf5a02c6f7f27a5cb2c'
gis = GIS()
data_cases = gis.content.get(daily_covid_county_service_id)
data_cases

<Item title:"Florida COVID19 Cases by County" type:Feature Layer Collection owner:Epidemiology.GIS>

In [11]:
query = data_cases.layers[0].query(where="1=1")
df_covid_cases = query.sdf
df_covid_cases.head()

,OBJECTID,OBJECTID_12_13,DEPCODE,COUNTY,COUNTYNAME,County_1,State,PUIsTotal,Age_0_4,Age_5_14,...,MonNow,Shape__Area,Shape__Length,GlobalID,NewPos,NewNeg,NewTested,NewPercPos,MedianAge,SHAPE
0,1,1,21,041,GILCHRIST,Gilchrist,FL,1476,39,84,...,750,9.204908e+08,148547.348006,cb6285a2-da2e-4583-aad5-1221776836ae,2,36,38,0.052632,41,"{""rings"": [[[529493.625602742, 647744.81243718..."
1,2,2,54,107,PUTNAM,Putnam,FL,6283,174,564,...,1607,2.142439e+09,232574.925675,ce511c5d-2638-44a3-b2f8-b13847543637,6,43,49,0.122449,40,"{""rings"": [[[633424.87518893, 650359.936015058..."
2,3,3,62,123,TAYLOR,Taylor,FL,2820,66,168,...,1563,2.685253e+09,267348.178950,541bd5a3-4453-499a-9896-f0edc5353d0b,11,28,39,0.282051,40,"{""rings"": [[[425905.96838189, 699605.625594474..."
3,4,4,46,091,OKALOOSA,Okaloosa,FL,15951,331,1138,...,10559,2.369223e+09,330553.160278,eb5d2450-a892-40fd-9ba4-9821a7438525,45,149,194,0.231959,40,"{""rings"": [[[170978.169305377, 740209.62491612..."
4,5,5,7,013,CALHOUN,Calhoun,FL,1562,33,120,...,1120,1.487627e+09,192207.890331,3b55aa95-5170-4caf-82a3-e99ad7459349,1,24,25,0.040000,44,"{""rings"": [[[310646.592981238, 733517.62524545..."


In [12]:
df_covid_cases_1 = df_covid_cases.drop(columns=['SHAPE'])
df_Case = df_covid_cases_1.sort_values(by=['DEPCODE'])
df_Case.head()

,OBJECTID,OBJECTID_12_13,DEPCODE,COUNTY,COUNTYNAME,County_1,State,PUIsTotal,Age_0_4,Age_5_14,...,EverMon,MonNow,Shape__Area,Shape__Length,GlobalID,NewPos,NewNeg,NewTested,NewPercPos,MedianAge
65,66,66,1,001,ALACHUA,Alachua,FL,26550,414,684,...,1769,1746,2.510756e+09,272135.471452,9eddb8fb-00dd-45f1-a09d-8e689049e28a,17,415,432,0.039352,24
16,17,17,2,003,BAKER,Baker,FL,3152,71,154,...,2221,2220,1.520246e+09,183737.354704,393e3bc9-079e-4ad6-9162-d77ee72feb59,5,27,32,0.156250,41
14,15,15,3,005,BAY,Bay,FL,11046,139,339,...,2028,2021,1.930771e+09,558412.990826,a66f3b06-8ee0-4dec-843e-f41d712aad2f,61,394,455,0.134066,42
39,40,40,4,007,BRADFORD,Bradford,FL,2968,69,160,...,1894,1863,7.767446e+08,167178.838253,588f1d06-056b-4af5-a2e7-a5366c833ba5,1,111,112,0.008929,46
41,42,42,5,009,BREVARD,Brevard,FL,32431,415,998,...,3251,3193,2.687644e+09,793218.333609,943db4a8-472c-444e-b81a-0e7cf9758ca2,145,934,1079,0.134384,44


In [13]:
cases_by_population_id = '09e0d01d8e8e46b6ae678c256d918178'
gis = GIS()
data_popu = gis.content.get(cases_by_population_id)
data_popu

<Item title:"Case Maps for COVID19" type:Feature Layer Collection owner:Epidemiology.GIS>

In [14]:
query_popu = data_popu.layers[0].query(where="1=1")
df_cases_popu = query_popu.sdf
df_cases_popu.head()

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,FID_1,CovidRate,Shape__Area,Shape__Length,SHAPE
0,1,Alachua,12001,263148,0,17,551,1041,11544,1,4387,2.509300e+09,262028.034556,"{""rings"": [[[529453.427928006, 647729.16620529..."
1,2,Baker,12003,27785,0,5,47,95,1885,2,6784,1.524801e+09,177464.398185,"{""rings"": [[[547611.939311098, 731650.13314648..."
2,3,Bay,12005,182482,0,61,381,595,7259,3,3978,2.285559e+09,233326.432744,"{""rings"": [[[208213.290884909, 697920.20090373..."
3,4,Bradford,12007,26979,0,1,36,56,1294,4,4796,7.782856e+08,161130.117599,"{""rings"": [[[552463.851341891, 658321.23030116..."
4,5,Brevard,12009,576808,0,145,869,1685,13371,5,2318,3.353781e+09,317459.583010,"{""rings"": [[[694017.58758627, 515464.701569436..."


In [15]:
df_cases_popu_1 = df_cases_popu.drop(columns=['SHAPE'])
df_PP = df_cases_popu_1.sort_values(by=['FID'])
df_PP.head()

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,FID_1,CovidRate,Shape__Area,Shape__Length
0,1,Alachua,12001,263148,0,17,551,1041,11544,1,4387,2.509300e+09,262028.034556
1,2,Baker,12003,27785,0,5,47,95,1885,2,6784,1.524801e+09,177464.398185
2,3,Bay,12005,182482,0,61,381,595,7259,3,3978,2.285559e+09,233326.432744
3,4,Bradford,12007,26979,0,1,36,56,1294,4,4796,7.782856e+08,161130.117599
4,5,Brevard,12009,576808,0,145,869,1685,13371,5,2318,3.353781e+09,317459.583010


In [16]:
print("Saving Florida_COVID19_Cases_by_County{}.xlsx in data/s".format(yearDate))
with pd.ExcelWriter('data/Florida_COVID19_Cases_by_County{}.xlsx'.format(yearDate)) as writer:  
    df_Case.to_excel(writer, sheet_name='Florida_COVID19_Cases_by_County')
    df_PP.to_excel(writer, sheet_name='Sheet1')

Saving Florida_COVID19_Cases_by_County111520.xlsx in data/s


In [17]:
df_case = df_Case[['OBJECTID','COUNTYNAME','County_1','State','NewPos','NewTested','PUIsTotal','CasesAll']].copy()
df_case.head()

,OBJECTID,COUNTYNAME,County_1,State,NewPos,NewTested,PUIsTotal,CasesAll
65,66,ALACHUA,Alachua,FL,17,432,26550,11651
16,17,BAKER,Baker,FL,5,32,3152,1893
14,15,BAY,Bay,FL,61,455,11046,7504
39,40,BRADFORD,Bradford,FL,1,112,2968,1296
41,42,BREVARD,Brevard,FL,145,1079,32431,13452


In [18]:
a = set(df_PP['NAME'].values)
b = set(df_case['County_1'].values)
a.difference(b)

{'A State'}

In [19]:
b.difference(a)

{'State'}

In [20]:
df_case.loc[df_case['County_1']=='State','County_1']='A State'

In [21]:
df_merge = pd.merge(df_PP, df_case, left_on=['NAME'],right_on= ['County_1'], how = 'outer')
df_merge.head(len(df_merge))

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,FID_1,...,Shape__Area,Shape__Length,OBJECTID,COUNTYNAME,County_1,State,NewPos,NewTested,PUIsTotal,CasesAll
0,1,Alachua,12001,263148,0,17,551,1041,11544,1,...,2.509300e+09,262028.034556,66,ALACHUA,Alachua,FL,17,432,26550,11651
1,2,Baker,12003,27785,0,5,47,95,1885,2,...,1.524801e+09,177464.398185,17,BAKER,Baker,FL,5,32,3152,1893
2,3,Bay,12005,182482,0,61,381,595,7259,3,...,2.285559e+09,233326.432744,15,BAY,Bay,FL,61,455,11046,7504
3,4,Bradford,12007,26979,0,1,36,56,1294,4,...,7.782856e+08,161130.117599,40,BRADFORD,Bradford,FL,1,112,2968,1296
4,5,Brevard,12009,576808,0,145,869,1685,13371,5,...,3.353781e+09,317459.583010,42,BREVARD,Brevard,FL,145,1079,32431,13452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,65,Wakulla,12129,31877,0,3,73,121,1439,65,...,1.633086e+09,234986.413040,41,WAKULLA,Wakulla,FL,3,66,3381,1448
65,66,Walton,12131,65858,0,18,210,651,2898,66,...,2.973057e+09,274860.715452,51,WALTON,Walton,FL,18,74,5230,3202
66,67,Washington,12133,24566,0,13,58,107,1344,67,...,1.595344e+09,237797.047191,53,WASHINGTON,Washington,FL,13,24,2397,1357
67,68,Unknown,99999,0,0,1,8,11,153,68,...,NaN,NaN,69,Unknown,Unknown,FL,1,825,3196,2076


In [22]:
df_merge_extract = df_merge.drop(columns=['OBJECTID','FID_1', 'COUNTYNAME'])
df_merge_extract.head()

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,Shape__Area,Shape__Length,County_1,State,NewPos,NewTested,PUIsTotal,CasesAll
0,1,Alachua,12001,263148,0,17,551,1041,11544,4387,2.509300e+09,262028.034556,Alachua,FL,17,432,26550,11651
1,2,Baker,12003,27785,0,5,47,95,1885,6784,1.524801e+09,177464.398185,Baker,FL,5,32,3152,1893
2,3,Bay,12005,182482,0,61,381,595,7259,3978,2.285559e+09,233326.432744,Bay,FL,61,455,11046,7504
3,4,Bradford,12007,26979,0,1,36,56,1294,4796,7.782856e+08,161130.117599,Bradford,FL,1,112,2968,1296
4,5,Brevard,12009,576808,0,145,869,1685,13371,2318,3.353781e+09,317459.583010,Brevard,FL,145,1079,32431,13452


In [23]:
df_merge_extract['P100K_Y'] = df_merge_extract['NewPos'] / df_merge_extract['Pop']*100000
df_merge_extract['P100K_7'] = df_merge_extract['Css7Dys'] / df_merge_extract['Pop']*100000
df_merge_extract['P100K_14'] = df_merge_extract['Css14Dy'] / df_merge_extract['Pop']*100000
df_merge_extract['T100K_Y'] = df_merge_extract['NewTested'] / df_merge_extract['Pop']*100000
df_merge_extract['PRate'] = df_merge_extract['NewPos'] / df_merge_extract['NewTested']
df_merge_extract['TestGoal'] = df_merge_extract['Css14Dy'] / 14 * 30

In [24]:
df_merge_drop_unknown = df_merge_extract.drop([67])
df_merge_drop_unknown

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,...,NewPos,NewTested,PUIsTotal,CasesAll,P100K_Y,P100K_7,P100K_14,T100K_Y,PRate,TestGoal
0,1,Alachua,12001,263148,0,17,551,1041,11544,4387,...,17,432,26550,11651,6.460243,209.387873,395.594874,164.166173,0.039352,2230.714286
1,2,Baker,12003,27785,0,5,47,95,1885,6784,...,5,32,3152,1893,17.995321,169.156019,341.911103,115.170056,0.156250,203.571429
2,3,Bay,12005,182482,0,61,381,595,7259,3978,...,61,455,11046,7504,33.427955,208.787716,326.059557,249.339661,0.134066,1275.000000
3,4,Bradford,12007,26979,0,1,36,56,1294,4796,...,1,112,2968,1296,3.706587,133.437118,207.568850,415.137700,0.008929,120.000000
4,5,Brevard,12009,576808,0,145,869,1685,13371,2318,...,145,1079,32431,13452,25.138348,150.656718,292.124936,187.063980,0.134384,3610.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,12127,527634,0,43,594,1258,14047,2662,...,43,676,35102,14158,8.149589,112.578037,238.422846,128.119113,0.063609,2695.714286
64,65,Wakulla,12129,31877,0,3,73,121,1439,4514,...,3,66,3381,1448,9.411174,229.005239,379.584026,207.045832,0.045455,259.285714
65,66,Walton,12131,65858,0,18,210,651,2898,4400,...,18,74,5230,3202,27.331531,318.867867,988.490388,112.362963,0.243243,1395.000000
66,67,Washington,12133,24566,0,13,58,107,1344,5471,...,13,24,2397,1357,52.918668,236.098673,435.561345,97.696003,0.541667,229.285714


In [25]:
df_merge_drop_unknown['P100K_Y'] =df_merge_drop_unknown['P100K_Y'].astype(int)
df_merge_drop_unknown['P100K_7'] =df_merge_drop_unknown['P100K_7'].astype(int)
df_merge_drop_unknown['P100K_14'] =df_merge_drop_unknown['P100K_14'].astype(int)
df_merge_drop_unknown['T100K_Y'] =df_merge_drop_unknown['T100K_Y'].astype(int)

In [26]:
df_merge_drop_unknown.head(len(df_merge_drop_unknown))

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,...,NewPos,NewTested,PUIsTotal,CasesAll,P100K_Y,P100K_7,P100K_14,T100K_Y,PRate,TestGoal
0,1,Alachua,12001,263148,0,17,551,1041,11544,4387,...,17,432,26550,11651,6,209,395,164,0.039352,2230.714286
1,2,Baker,12003,27785,0,5,47,95,1885,6784,...,5,32,3152,1893,17,169,341,115,0.156250,203.571429
2,3,Bay,12005,182482,0,61,381,595,7259,3978,...,61,455,11046,7504,33,208,326,249,0.134066,1275.000000
3,4,Bradford,12007,26979,0,1,36,56,1294,4796,...,1,112,2968,1296,3,133,207,415,0.008929,120.000000
4,5,Brevard,12009,576808,0,145,869,1685,13371,2318,...,145,1079,32431,13452,25,150,292,187,0.134384,3610.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,12127,527634,0,43,594,1258,14047,2662,...,43,676,35102,14158,8,112,238,128,0.063609,2695.714286
64,65,Wakulla,12129,31877,0,3,73,121,1439,4514,...,3,66,3381,1448,9,229,379,207,0.045455,259.285714
65,66,Walton,12131,65858,0,18,210,651,2898,4400,...,18,74,5230,3202,27,318,988,112,0.243243,1395.000000
66,67,Washington,12133,24566,0,13,58,107,1344,5471,...,13,24,2397,1357,52,236,435,97,0.541667,229.285714


In [27]:
gdf_county_polygon = gpd.read_file('shp/Florida_Counties.shp')
gdf_county_polygon.head()

,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,21,041,Gilchrist,2000-05-16,9.908353e+09,4.873000e+05,"POLYGON ((-82.65814 29.83028, -82.65803 29.822..."
1,2,54,107,Putnam,2000-05-16,2.305869e+10,7.629677e+05,"POLYGON ((-81.58084 29.83956, -81.57893 29.835..."
2,3,62,123,Taylor,2000-05-16,2.891747e+10,8.772527e+05,"MULTIPOLYGON (((-83.73037 30.30358, -83.70368 ..."
3,4,46,091,Okaloosa,2000-05-16,2.562159e+10,1.087058e+06,"MULTIPOLYGON (((-86.39159 30.64970, -86.39175 ..."
4,5,7,013,Calhoun,2000-05-16,1.604809e+10,6.313440e+05,"POLYGON ((-84.93266 30.60637, -84.93287 30.605..."


In [28]:
a = set(df_merge_drop_unknown['County_1'].values)
b = set(gdf_county_polygon['COUNTYNAME'].values)
a.difference(b)

{'A State'}

In [29]:
b.difference(a)

set()

In [30]:
df_merge_poly = pd.merge(df_merge_drop_unknown, gdf_county_polygon, left_on = ['County_1'],right_on= ['COUNTYNAME'], how = 'outer')
df_merge_poly.head(len(df_merge_poly))

,FID,NAME,GEOID,Pop,Pop_MOE,CssYstr,Css7Dys,Css14Dy,CassAll,CovidRate,...,PRate,TestGoal,OBJECTID,DEPCODE,COUNTY,COUNTYNAME,DATESTAMP,ShapeSTAre,ShapeSTLen,geometry
0,1,Alachua,12001,263148,0,17,551,1041,11544,4387,...,0.039352,2230.714286,66.0,1.0,001,Alachua,2000-05-16,2.702324e+10,8.927266e+05,"POLYGON ((-82.41869 29.92298, -82.41825 29.922..."
1,2,Baker,12003,27785,0,5,47,95,1885,6784,...,0.156250,203.571429,17.0,2.0,003,Baker,2000-05-16,1.636216e+10,6.027173e+05,"POLYGON ((-82.05012 30.36233, -82.04949 30.316..."
2,3,Bay,12005,182482,0,61,381,595,7259,3978,...,0.134066,1275.000000,15.0,3.0,005,Bay,2000-05-16,2.084232e+10,1.834701e+06,"MULTIPOLYGON (((-85.38551 30.50018, -85.38650 ..."
3,4,Bradford,12007,26979,0,1,36,56,1294,4796,...,0.008929,120.000000,40.0,4.0,007,Bradford,2000-05-16,8.359863e+09,5.483267e+05,"POLYGON ((-82.04939 30.14322, -82.04938 30.143..."
4,5,Brevard,12009,576808,0,145,869,1685,13371,2318,...,0.134384,3610.714286,42.0,5.0,009,Brevard,2000-05-16,2.893695e+10,2.600567e+06,"MULTIPOLYGON (((-80.80666 28.61413, -80.80790 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
63,64,Volusia,12127,527634,0,43,594,1258,14047,2662,...,0.063609,2695.714286,25.0,64.0,127,Volusia,2000-05-16,3.364565e+10,1.837936e+06,"MULTIPOLYGON (((-81.09705 29.35180, -81.09292 ..."
64,65,Wakulla,12129,31877,0,3,73,121,1439,4514,...,0.045455,259.285714,41.0,65.0,129,Wakulla,2000-05-16,1.702251e+10,9.167339e+05,"MULTIPOLYGON (((-84.41197 30.30244, -84.37858 ..."
65,66,Walton,12131,65858,0,18,210,651,2898,4400,...,0.243243,1395.000000,51.0,66.0,131,Walton,2000-05-16,2.926099e+10,1.216747e+06,"POLYGON ((-86.03523 30.98825, -86.03532 30.982..."
66,67,Washington,12133,24566,0,13,58,107,1344,5471,...,0.541667,229.285714,53.0,67.0,133,Washington,2000-05-16,1.721229e+10,8.010119e+05,"POLYGON ((-85.43543 30.56727, -85.46843 30.567..."


In [31]:
result_folder = 'results/{}'.format(yearDate)
if not os.path.exists(result_folder):
    os.makedirs(result_folder)

In [32]:
shpfile_name = "covid_impact_poly"+ yearDate
print("Saving {}.shp in results/".format(shpfile_name))
gdf_merge_poly = gpd.GeoDataFrame(df_merge_poly, geometry='geometry',crs="EPSG:4326")
gdf_merge_poly.to_file(os.path.join(result_folder,"{}.shp".format(shpfile_name)))

Saving covid_impact_poly111520.shp in results/


In [33]:
print("Saving {}.zip in publish/".format(shpfile_name))
with ZipFile(os.path.join('publish', '{}.zip'.format(shpfile_name)), 'w') as zipObj:
    for file in os.listdir(result_folder):
        if shpfile_name in file:
            file_path = os.path.join(result_folder, file)
            print(file)
            try:
                zipObj.write(file_path, os.path.basename(file_path))
            except OSError as err:
                print("OS error: {0}".format(err))   

Saving covid_impact_poly111520.zip in publish/
covid_impact_poly111520.prj
covid_impact_poly111520.cpg
covid_impact_poly111520.shp
covid_impact_poly111520.shx
covid_impact_poly111520.dbf


In [34]:
print(str(datetime.now()) + " Done with 1_COVID_Daily_Cases_Process")

2020-11-15 12:55:12.870688 Done with 1_COVID_Daily_Cases_Process
